# Web Scraping

In [6]:
import urllib
from bs4 import BeautifulSoup
url = "http://web.mta.info/developers/turnstile.html"
u = urllib.request.urlopen(url)
html = u.read()

soup = BeautifulSoup(html, "html.parser")
links = soup.find_all('a')

count = 0

for link in links:
    if '1706' in str(link.get('href')):
        print("http://web.mta.info/developers/" + link.get('href'))
        urllib.request.urlretrieve("http://web.mta.info/developers/" + link.get('href'), "turnstile_100617" + str(count) + ".txt")
        count = count + 1

http://web.mta.info/developers/data/nyct/turnstile/turnstile_170624.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_170617.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_170610.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_170603.txt


In [1]:
import pandas as pd
filename = "turnstile_data_master_with_weather.csv"
weather_data = pd.read_csv(filename)

weather_data.head()

Unnamed: 0  UNIT       DATEn     TIMEn  Hour    DESCn  ENTRIESn_hourly  \
0           0  R001  2011-05-01  01:00:00     1  REGULAR              0.0   
1           1  R001  2011-05-01  05:00:00     5  REGULAR            217.0   
2           2  R001  2011-05-01  09:00:00     9  REGULAR            890.0   
3           3  R001  2011-05-01  13:00:00    13  REGULAR           2451.0   
4           4  R001  2011-05-01  17:00:00    17  REGULAR           4400.0   

   EXITSn_hourly  maxpressurei  maxdewpti  ...  meandewpti  meanpressurei  \
0            0.0         30.31       42.0  ...        39.0          30.27   
1          553.0         30.31       42.0  ...        39.0          30.27   
2         1262.0         30.31       42.0  ...        39.0          30.27   
3         3708.0         30.31       42.0  ...        39.0          30.27   
4         2501.0         30.31       42.0  ...        39.0          30.27   

   fog  rain  meanwindspdi  mintempi  meantempi  maxtempi  precipi  thunder  
0  0.0   0.0           5.0      50.0       60.0      69.0      0.0      0.0  
1  0.0   0.0           5.0      50.0       60.0      69.0      0.0      0.0  
2  0.0   0.0           5.0      50.0       60.0      69.0      0.0      0.0  
3  0.0   0.0           5.0      50.0       60.0      69.0      0.0      0.0  
4  0.0   0.0           5.0      50.0       60.0      69.0      0.0      0.0  

[5 rows x 22 columns]

In [2]:
import pandasql

def num_rainy_days(filename):
    weather_data = pd.read_csv(filename)
    
    q = """
    SELECT COUNT(*)
    FROM weather_data
    WHERE rain=1;
    """
    
    rainy_days = pandasql.sqldf(q.lower(), locals())
    return rainy_days

csv_data = "turnstile_data_master_with_weather.csv"
print(num_rainy_days(csv_data))

   count(*)
0     44104


In [4]:
import sys

def mapper():
    for line in sys.stdin:
        data = line.strip().split(',')
        if len(data) == 22:
            Unnamed,UNIT,DATEn,TIMEn,Hour,DESCn,ENTRIESn_hourly,EXITSn_hourly,maxpressurei,maxdewpti,mindewpti,minpressurei,meandewpti,meanpressurei,fog,rain,meanwindspdi,mintempi,meantempi,maxtempi,precipi,thunder = data
            print("{0}\t{1}".format(UNIT,ENTRIESn_hourly))
            
sys.stdin = open('turnstile_data_master_with_weather.csv')
sys.stdout = open('mapper_result.txt','w')
mapper()

In [8]:
import sys
def reducer():
    entriesTotal = 0
    oldKey = None
    for line in sys.stdin:
        data = line.strip().split('\t')
        if len(data) != 2:
            continue
        thisKey, thisEntry = data
        if oldKey and oldKey != thisKey:
            print("{0}\t{1}".format(oldKey, entriesTotal))
            entriesTotal = 0
        oldKey = thisKey
        try:
            entriesTotal += float(thisEntry)
        except:
            print(thisEntry)
    if oldKey != None:
        print("{0}\t{1}".format(oldKey, entriesTotal))
        
sys.stdin = open('mapper_result.txt','r')
sys.stdout = open('reducer_result.txt','w')
reducer()